In [1]:
import json
from extract import POLOCMBASELINE
import pandas as pd
from traces import *
from observation import *

In [2]:
from os import read

from extract import polocm


def read_json_file(read_only):
    json_file_path = "../data/no_obj_limit/traces_plan_r1.json"
    with open(json_file_path, 'r') as file:
        data = json.load(file)
        return data

In [3]:


    
from ast import Dict
from extract.model import Model
import os
from planner import PseudoPlanner

def polocm_baseline (obs_tracelist: ObservedTraceList, domain_name,debug, exe_debug= False, ):
    try:
        model, _, runtime = POLOCMBASELINE(obs_tracelist,sort_knowledge=False, debug=debug)
        filename = domain_name + ".pddl"
        file_path = os.path.join("../output/test_pb", filename)
        tmp_path = os.path.join("../output/test_pb/tmp", filename)
    
    
        if isinstance(model, Model):
            model.to_pddl(domain_name, domain_filename=file_path, problem_filename=tmp_path, )
    except Exception as e:
        print("error to pddl", e)
        return 0

    exe = get_executability(obs_tracelist, domain_filename=file_path, debug=exe_debug)
    return exe

def get_executability(obs_tracelist, domain_filename=None, planner=None, debug=False):
    if (planner and not domain_filename):
        pp = planner
        
    elif(domain_filename and not planner):
        pp = PseudoPlanner(domain_filename, 'first_fail')
    else:
        raise Exception("Either domain_filename or planner should be provided")

    trace = obs_tracelist[0]
    actions = [step.action for step in trace]
    try: 
        res= pp.check_executability(actions,debug=debug)
    except Exception as e:
        raise Exception("Error in checking executability")
        

    return res




In [4]:

def run_single_experiment(learning_obj, dod, debug):
    domain = learning_obj['domain']
    raw_traces = learning_obj['traces']

    try:
        traces = []
        for raw_trace in raw_traces:
            steps = []
            for i, raw_step in enumerate(raw_trace):
                action_name = raw_step['action']
                obj_names = raw_step['objs']
                objs = []
                for obj in obj_names:
                    obj_name, obj_type = obj.split("?")
                    objs.append(PlanningObject(obj_type, obj_name))
                action = Action(action_name, objs)
                step = Step(State(), action, i)
                steps.append(step)
            trace = Trace(steps)
            traces.append(trace)

        tracelist = TraceList(traces)
        obs_tracelist = tracelist.tokenize(ActionObservation, ObservedTraceList)
     
        error_rate = 0
        
        poats = learning_obj['pos']
        i = int((dod * 10)-1)
        poat = poats[i]
        actual_dod = poat['actual_dod']
        inds = poat['traces_inx']
        pos = poat['po']
        po_traces = []
        
        for i,trace in enumerate(tracelist):
            po = pos[i]
            ind = inds[i]
            po_steps =[]
            for j,po_step_ind in enumerate(ind):
                ori_step = trace[po_step_ind]
                po_step = PartialOrderedStep(ori_step.state, ori_step.action, ori_step.index, po[j])
                po_steps.append(po_step)
            po_traces.append(PartialOrderedTrace(po_steps, actual_dod))
        
        po_tracelist = TraceList(po_traces)
        obs_po_tracelist = po_tracelist.tokenize(PartialOrderedActionObservation, ObservedPartialOrderTraceList)

        executability = polocm_baseline(obs_po_tracelist, domain, debug)

  
    except Exception as e:
        return -1
    return executability


In [5]:
read_only = "satellite"
# read_only = None
data = read_json_file(read_only)




In [8]:
debug = {'pstep1': True,'2step0': True, 'sorts': True}
example = data[289]

exe = run_single_experiment(example, 0.1, debug= debug)

Sorts:
{'table1': 1, 'table3': 1, 'table2': 1, 'kitchen': 1, 'sandw8': 2, 'sandw1': 2, 'sandw2': 2, 'sandw4': 2, 'sandw5': 2, 'sandw3': 2, 'sandw10': 2, 'sandw7': 2, 'sandw6': 2, 'sandw9': 2, 'child3': 3, 'child10': 3, 'child1': 3, 'child5': 3, 'child7': 3, 'child9': 3, 'child4': 3, 'child6': 3, 'child8': 3, 'child2': 3, 'bread3': 4, 'bread4': 4, 'bread8': 4, 'bread1': 4, 'bread9': 4, 'bread2': 4, 'bread5': 4, 'bread6': 4, 'bread7': 4, 'bread10': 4, 'content9': 5, 'content5': 5, 'content8': 5, 'content3': 5, 'content4': 5, 'content2': 5, 'content6': 5, 'content7': 5, 'content1': 5, 'content10': 5, 'tray1': 6, 'zero': 0}



# Step 1

## Obj PO traces:


### Trace **0**


+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| 0                        | 1                                                                                                                                                                                                                                                                                                                                                                 |
|--------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

...

Sort.5 AML:


''

Sort.6 AML:


,serve_sandwich.3,serve_sandwich_no_gluten.3,move_tray.1,put_on_tray.2
serve_sandwich.3,41,44,40,8
serve_sandwich_no_gluten.3,43,41,32,8
move_tray.1,49,34,13,16
put_on_tray.2,22,27,38,211


sort 0
[serve_sandwich_0 serve_sandwich_no_gluten_0 move_tray_0 put_on_tray_0
 make_sandwich_0 make_sandwich_no_gluten_0]
----------------------------------------
sort 1
[serve_sandwich_4 serve_sandwich_no_gluten_4 move_tray_3 move_tray_2]
----------------------------------------
sort 2
[make_sandwich_1 put_on_tray_1 serve_sandwich_1 serve_sandwich_no_gluten_1
 make_sandwich_no_gluten_1]
----------------------------------------
sort 3
[]
----------------------------------------
sort 4
[]
----------------------------------------
sort 5
[]
----------------------------------------
sort 6
[serve_sandwich_3 serve_sandwich_no_gluten_3 move_tray_1 put_on_tray_2]
----------------------------------------
sort 0
[{serve_sandwich_0, serve_sandwich_no_gluten_0, move_tray_0, make_sandwich_no_gluten_0, put_on_tray_0, make_sandwich_0}]
sort 1
[frozenset({move_tray_2, move_tray_3}), {serve_sandwich_no_gluten_4, serve_sandwich_4, move_tray_2, move_tray_3}]
sort 2
[{make_sandwich_no_gluten_1, serve_sand

In [7]:
# statics = ['childsnack', 'floortile', 'miconic', 'rovers', 'satellite', 'sokoban', 'spanner', 'transport']
# headers = ["domain", "exe", "zero_sort_removed", "static_conditions"]
# for i,learn_obj in enumerate(data):
    
#     exe = run_single_experiment(learn_obj, 0.1, None)